# Hi SMSQ team!
This Colab file will be used to structure our recomender system!

It is built in 3 parts:


1.  Text political bias network

![recomendation diagram](https://i.ibb.co/tMQ5GQB/Blank-diagram.png)


In [ ]:
!ls

# Handeling Imports

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
import re
import string

#Preprocessing:
!pip install tensorflow_transform
import tensorflow_transform as tft
import tensorflow_hub as hub



#Ranking:
# !pip install -q tensorflow_ranking tensorflow-serving-api

# import tensorflow_ranking as tfr
# from tensorflow_serving.apis import input_pb2

#Model:
from keras.regularizers import l1, l2
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras import backend
from keras import optimizers

# Track Recourse usage

In [ ]:
from urllib.request import urlopen
exec(urlopen("http://colab-monitor.smankusors.com/track.py").read())
_colabMonitor = ColabMonitor().start()

In [ ]:
tf.config.list_physical_devices()



#Part 1 - Text political bias network

## Getting "all the news" dataset

In [ ]:
!wget https://www.dropbox.com/s/cn2utnr5ipathhh/all-the-news-2-1.zip -q --show-progress -O atn.zip

In [ ]:
!unzip atn.zip

In [ ]:
 !wget https://gist.githubusercontent.com/nsfyn55/605783ac8de36f361fb10ef187272113/raw/2a115dc6d4d8c3b14f84b12d784363ea36935e9d/media-bias-scrubbed-results.csv -q -O l.csv

## Parsing Data

In [ ]:
import pandas as pd
import numpy as np

# Make numpy values easier to read.
np.set_printoptions(precision=3, suppress=True)

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

In [ ]:
news_csv_ds = tf.data.experimental.make_csv_dataset(
    "all-the-news-2-1.csv",
    batch_size=8,
    label_name='publication',
    select_columns=["article", "publication"],
    shuffle=True,
    )

bias_scored = pd.read_csv("l.csv")

# bias_scored['url'] = bias_scored['url'].apply(lambda x: "/".join(x.split("/")[:3]))

bias_scored.set_index("site_name")

# bias_scored = list(map(lambda x:x.lower(), list(bias_scored["site_name"])))

bias_scored["site_name"] = bias_scored["site_name"].map(lambda x: re.sub(r'\([^)]*\)', '', x).strip().lower())
# bias_scored = np.array(list(map(lambda x:re.sub(r'\([^)]*\)', '', x).strip() ,bias_scored)))


bias_scored = bias_scored.drop_duplicates(subset='site_name')

ratings = bias_scored["bias_rating"]
bias_scored = bias_scored["site_name"]
table = tf.lookup.StaticHashTable(
    tf.lookup.KeyValueTensorInitializer(bias_scored, ratings), default_value=0)


In [ ]:
ratings

In [ ]:
#define process function for datapoint to convert publisher name to bias value

# @tf.function
# def m(x):
#   t =  table.lookup(x)
#   return t
@tf.function
def tf_processe(article, label):
  return [article['article'], tf.map_fn(lambda x: table.lookup(tf.strings.lower(x)), label, fn_output_signature=tf.int64)]

In [ ]:
news_csv_ds = news_csv_ds.map(tf_processe) #apply the mapping function on the dataset (actual mapping will happen in realtime using graph api)

In [ ]:
test_dataset = news_csv_ds.take(100) #generate test & train datasets
train_dataset = news_csv_ds.skip(100)

In [ ]:
iter(test_dataset).next()

## Configuring Model

In [ ]:
!pip install tensorflow_text
import tensorflow_text as text  # Registers the ops.

In [ ]:
preprocessor = "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3"

bert_module = "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3"


In [ ]:

def build_classifier_model():
  text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='article')
  preprocessing_layer = hub.KerasLayer(preprocessor, name='preprocessing') #arguments=dict(seq_length=512))
  encoder_inputs = preprocessing_layer(text_input)
  encoder = hub.KerasLayer(bert_module, trainable=True, name='BERT_encoder')
  outputs = encoder(encoder_inputs)
  net = outputs['pooled_output']
  net = tf.keras.layers.Dropout(0.1)(net)
  net = tf.keras.layers.Dense(1, name='classifier')(net)
  return tf.keras.Model(text_input, net)



In [ ]:
bias_NN = build_classifier_model();
bias_NN.summary()
tf.keras.utils.plot_model(bias_NN)

#print summary


checkpoint_path = "logs/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1, save_freq= 10000)


In [ ]:
!pip install -q tf-models-official
from official.nlp import optimization  # to create AdamW optmizer


In [ ]:
 steps_per_epoch = 325000 #approx

In [ ]:
#compile model:
loss = tf.keras.losses.MeanSquaredError()
# metrics = tf.metrics.BinaryAccuracy()


epochs = 1
# steps_per_epoch = tf.data.experimental.cardinality(train_dataset).numpy()
print("steps per epoch: "+str(steps_per_epoch))
num_train_steps = steps_per_epoch * epochs
num_warmup_steps = int(0.1*num_train_steps)
print("num of warmup steps: "+str( num_warmup_steps))
init_lr = 3e-5
optimizer = optimization.create_optimizer(init_lr=init_lr,
                                          num_train_steps=num_train_steps,
                                          num_warmup_steps=num_warmup_steps,
                                          optimizer_type='adamw')

bias_NN.compile(optimizer=optimizer,
                         loss=loss)


In [ ]:
!ls /kaggle/working

## Training the model

In [ ]:
!pip install wandb #install weights & biases from pip

In [ ]:
import wandb
wandb.init(project="smsq", sync_tensorboard=True) #config wandb project

In [ ]:
log_dir = "logs/"  #config tensorboard
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, update_freq=100)

In [ ]:
from wandb.keras import WandbCallback # import wandb callback

In [ ]:
#train model
hist = bias_NN.fit(train_dataset, epochs=epochs, validation_data=test_dataset, callbacks=[cp_callback, tensorboard_callback, WandbCallback()])#,

In [ ]:
bias_NN.save('models/modelX')
!zip -r d.zip models

In [ ]:
iter(train_dataset).next()

In [ ]:
# Save the entire model as a SavedModel.
bias_NN.save('modelY')

In [ ]:
!zip -r modelY modelY

In [ ]:
bias_NN(tf.constant(["WASHINGTON — Led by loyalists who embrace former President Donald J. Trump’s baseless claims of a stolen election, Republicans in state legislatures nationwide are mounting extraordinary efforts to change the rules of voting and representation — and enhance their own political clout. At the top of those efforts is a slew of bills raising new barriers to casting votes, particularly the mail ballots that Democrats flocked to in the 2020 election. But other measures go well beyond that, including tweaking Electoral College and judicial election rules for the benefit of Republicans; clamping down on citizen-led ballot initiatives; and outlawing private donations that provide resources for administering elections, which were crucial to the smooth November vote. And although the decennial redrawing of political maps has been pushed to the fall because of delays in delivering 2020 census totals, there are already signs of an aggressive drive to further gerrymander political districts, particularly in states under complete Republican control. The national Republican Party joined the movement this past week by setting up a Committee on Election Integrity to scrutinize state election laws, echoing similar moves by Republicans in a number of state legislatures. Republicans have long thought — sometimes quietly, occasionally out loud — that large turnouts, particularly in urban areas, favor Democrats, and that Republicans benefit when fewer people vote. But politicians and scholars alike say that this moment feels like a dangerous plunge into uncharted waters. Thanks for reading The Times. Subscribe to The Times The avalanche of legislation also raises fundamental questions about the ability of a minority of voters to exert majority control in American politics, with Republicans winning the popular vote in just one of the last eight presidential elections but filling six of the nine seats on the Supreme Court. The party’s battle in the past decade to raise barriers to voting, principally among minorities, young people and other Democrat-leaning groups, has been waged under the banner of stopping voter fraud that multiple studies have shown barely exists. “The typical response by a losing party in a functioning democracy is that they alter their platform to make it more appealing,” Kenneth Mayer, an expert on voting and elections at the University of Wisconsin-Madison, said. “Here the response is to try to keep people from voting. It’s dangerously antidemocratic.” Editors’ Picks The Way We Worked Out The Goal: A One-Bedroom With a View in Queens for Less Than $650,000. Which Option Would You Choose? ‘I Paid It No Mind and Kept My Head Low, Glued to My Music’ Continue reading the main story ImageThe most conspicuous of the Republicans’ efforts are a slew of bills raising barriers to casting votes, particularly mail-in ballots. The most conspicuous of the Republicans’ efforts are a slew of bills raising barriers to casting votes, particularly mail-in ballots.Credit...Robert Nickelsberg for The New York Times Consider Iowa, a state that has not been a major participant in the past decade’s wars over voting and election rules. The November election saw record turnout and little if any reported fraud. Republicans were the state’s big winners, including in the key races for the White House and Senate. Yet, in a vote strictly along party lines, the State Legislature voted this past week to cut early voting by nine days, close polls an hour earlier and tighten rules on absentee voting, as well as strip the authority of county auditors to decide how election rules can best serve voters. State Senator Jim Carlin, a Republican who recently announced his candidacy for the U.S. Senate, made the party’s position clear during the floor debate: “Most of us in my caucus and the Republican caucus believe the election was stolen,” he said. State Senator Joe Bolkcom, a Democrat, said that served as justification for a law that created “a voting system tailored to the voting tendency of older white Republican voters.” “They’ve convinced all their supporters of the big lie. They don’t see any downside in this,” he said in an interview. “It’s a bad sign for the country. We’re not going to have a working democracy on this path.” The issues are particularly stark because fresh restrictions would disproportionately hit minorities just as the nation is belatedly reckoning with a racist past, said Lauren Groh-Wargo, the chief executive of the voting advocacy group Fair Fight Action. The Republican push comes as the rules and procedures of American elections increasingly have become a central issue in the nation’s politics. The Brennan Center for Justice, a liberal-leaning law and justice institute at New York University, counts 253 bills in 43 states that seek to tighten voting rules. At the same time, 704 bills have been introduced with provisions to improve access to voting. The push also comes as Democrats in Congress are attempting to pass federal legislation that would tear down barriers to voting, automatically register new voters and outlaw gerrymanders, among many other measures. Some provisions, such as a prohibition on restricting a voter’s ability to cast a mail ballot, could undo some of the changes being proposed in state legislatures. Such legislation, combined with the renewed enforcement of federal voting laws, could counter some Republican initiatives in the 23 states where the party controls the legislature and governor’s office. But neither that Democratic proposal nor a companion effort to enact a stronger version of the 1965 Voting Rights Act stands any chance of passing unless Democrats modify or abolish Senate rules allowing filibusters. It remains unclear whether the party has either the will or the votes to do that. Image “Most of us in my caucus and the Republican caucus believe the election was stolen,” State Senator Jim Carlin of Iowa said of Donald J. Trump’s loss to President Biden. “Most of us in my caucus and the Republican caucus believe the election was stolen,” State Senator Jim Carlin of Iowa said of Donald J. Trump’s loss to President Biden.Credit...Anna Moneymaker for The New York Times On the legal front, the Supreme Court will hear arguments on Tuesday in an Arizona election lawsuit that turns on the enforcement of Section 2 of the Voting Rights Act. That section is the government’s main remaining weapon against discriminatory voting practices after the court struck down another provision in 2013 that gave the Justice Department broad authority over voting in states with histories of discrimination. Those who back the Republican legislative efforts say they are needed to restore flagging public confidence in elections and democracy, even as some of them continue to attack the system as corrupt. In Arizona, Pennsylvania and Wisconsin, for example, the chairs of House election committees refused for weeks or months to affirm that President Biden won the election. The chairs in Pennsylvania and Wisconsin urged U.S. House members or former Vice President Mike Pence to oppose the presidential electors certified after Mr. Biden won those states’ votes. Some respected Republican lawmakers reject charges that election proposals are bad-faith attempts to advance Republican power. “These are really big tweaks. I get that,” said State Senator Kathy Bernier, who heads an election committee in Wisconsin. “But we do this routinely every session.” Ms. Bernier said the party’s election-law bills, two of which would strengthen ID requirements for absentee ballots and limit ballot drop boxes to one per municipality, were honest efforts to make voting more secure. That said, proposals in many states have little or nothing to do with that goal. Georgia Republicans would sharply limit early voting on Sundays, when many Black voters follow church services with “souls to the polls” bus rides to cast ballots. On Friday, a State Senate committee approved bills to end no-excuse absentee voting and automatic voter registration at motor vehicle offices. Iowa’s legislation, passed this past week, also shortens the windows to apply for absentee ballots and petition for satellite polling places deployed at popular locations like college campuses and shopping centers. Bills in some states to outlaw private donations to fund elections are rooted in the unproven belief, popular on the right, that contributions in 2020 were designed to increase turnout in Democratic strongholds. The nonprofit Center for Technology and Civic Life distributed the $400 million that the Facebook founder Mark Zuckerberg and his wife, Priscilla Chan, donated to underwrite coronavirus protective equipment, polling place rentals, drop boxes and other election needs. Unsurprisingly, some of the most vigorous efforts by Republicans are in swing states where last year’s races for national offices were close. Image An early voting site for Georgia’s Senate runoff at the Mercedes-Benz Stadium in Atlanta in December.&nbsp; An early voting site for Georgia’s Senate runoff at the Mercedes-Benz Stadium in Atlanta in December. Credit...Erik S Lesser/EPA, via Shutterstock Republicans in Georgia, which Mr. Biden won by roughly 12,000 votes, lined up this week behind a State Senate bill that would require vote-by-mail applications to be made under oath, with some requiring an additional ID and a witness signature. Arizona Republicans are backing bills to curtail the automatic mailing of absentee ballots to voters who skip elections, and to raise to 60 percent the share of votes required to pass most citizen ballot initiatives. Legislatures in at least five other Republican-run states are also considering bills making it harder to propose or pass citizen-led initiatives, which often involve issues like redistricting or tax hikes where the party supports the status quo. And that is not all: One Arizona Republican has proposed legislation that would allow state lawmakers to ignore the results of presidential elections and decide themselves which candidate would receive the state’s electoral votes. In Wisconsin, where gerrymanders of the State Legislature have locked in Republican control for a decade, the Legislature already has committed at least $1 million for law firms to defend its redistricting of legislative and congressional seats this year. The gerrymander proved impregnable in November; Democrats received 46 percent of the statewide vote for State Assembly seats and 47 percent of the State Senate vote, but won only 38 percent of seats in the Assembly and 36 percent in the Senate. In New Hampshire, where Republicans took full control of the Legislature in November, the party chairman, Stephen Stepanek, has indicated he backs a gerrymander of the state’s congressional map to “guarantee” that at least one of the state’s two Democrats in the U.S. House would not win re-election. “Elections have consequences,” he told the news outlet Seacoastonline. He did not respond to a request for comment. And in Nebraska, one of only two states that award electoral votes in presidential contests by congressional district, conservatives have proposed to switch to a winner-take-all model after Mr. Biden captured an electoral vote in the House district containing Omaha, the state’s sole Democratic bastion. Conversely, some New Hampshire Republicans would switch to Nebraska’s current Electoral College model instead of the existing winner-take-all method. That would appear to help Republicans in a state where Democrats have won the past five presidential elections. Pennsylvania’s Legislature is pushing a gerrymander-style apportionment of State Supreme Court seats via a constitutional amendment that would elect justices by regions rather than statewide. That would dismantle a lopsided Democratic majority on the court by creating judicial districts in more conservative rural reaches. Many Republicans argue — and some election experts at times agree — that fears about restrictive election laws among Democrats and civil liberties advocates can be overblown. Republicans point to record turnout in November as proof that restrictive laws do not suppress votes. Ms. Bernier of Wisconsin, for example, said she saw little problem with a bill that would allot one ballot drop box for voters in towns like New Berlin, with 40,000 residents, and one for voters in Milwaukee, with 590,000 residents. There were no drop boxes at all, she noted, until state officials made an emergency exception during the pandemic. “The Legislature could say that no drop boxes are necessary at all,” she said. Nathaniel Persily, a Stanford University political scientist and election expert, said he disagreed. Presidential elections always draw more voters, he said, but the grunt work of democracy often occurs in off-year votes for lesser offices where interest is lower. In those elections, “if there are barriers placed in the way of voters, they’re not going to turn out,” he said. Mike Noble, a Phoenix public-opinion expert, questioned whether the Arizona Legislature’s Trumpian anti-fraud agenda has political legs, even though polls show a level of Republican belief in Mr. Trump’s stolen election myth that he calls “mind-boggling.” Republicans who consider themselves more moderate make up about a third of the party’s support in Arizona, he said, and they are far less likely to believe the myth. And they may be turned off by a Legislature that wants to curtail absentee ballot mailings in a state where voters — especially Republicans — have long voted heavily by mail. “I don’t see how a rational person would see where the benefit is,” he said. Some other Republicans apparently agree. In Kentucky, which has some of the nation’s strictest voting laws, the solidly Republican State House voted almost unanimously on Friday to allow early voting, albeit only three days, and online applications for absentee ballots. Both were first tried during the pandemic and, importantly, were popular with voters and county election officials. If that kind of recognition of November’s successes resonated in other Republican states, Mr. Persily and another election scholar, Charles Stewart III of the Massachusetts Institute of Technology, wrote in a recent study, it could bode well for easing the deep divisions over future election rules. If the stolen election myth continues to drive Republican policy, Mr. Persily said, it could foretell a future with two kinds of elections in which voting rights, participation and faith in the results would be significantly different, depending on which party had written the rules. “Those trajectories are on the horizon,” he said. “Some states are adopting a blunderbuss approach to regulating voting that is only distantly related to fraud concerns. And it could mean massive collateral damage for voting rights.”"]))

In [ ]:
bias_NN(tf.constant(["Former Acting Director of National Intelligence Richard Grenell took on embattled California Gov. Gavin Newsom, who faces a potential recall, and the D.C. political establishment during a speech at the Conservative Political Action Conference (CPAC) on Saturday. “In America, I think there’s a real value in seriously considering term limits for our elected officials,” Grenell said to loud cheering. “Many who have become more ingrained in Washington than in the fabric of their own hometowns. This is a fight between the insiders and outsiders, between Washington, DC, and the rest of America,” he said. Grenell then pivoted to California, heightening speculation he may run against California Gov. Gavin Newsom. “Of course if things get really bad with elected officials, there is always the option to recall them,” he said. “If you want the best case possible for a recall campaign, take a look at my home state of California.” He added: California used to be Reagan country. The shining example of business innovation and middle class success. But now when you think of California, you think of out of control wildfires, of rolling blackouts, of schools still closed, of shuttered businesses, of bans on fracking and wealthy people jumping the vaccine line. He blasted Newsom for purchasing $1 billion in masks and other personal protective equipment from a Chinese company “when American companies with the same equipment were based in California.” “In my three decades in American politics, I have never seen a better case for a recall than there is right now in California,” he said. He then hinted at a potential run against Newsom. “And of course if a public official is still failing to deliver on their promises and if you can’t limit their term or recall them in time. There’s always one other option. You can run against them yourself,” he said. Grenell was the first openly-gay member of a presidential cabinet. He grew up in California and Michigan. He served as the spokesman for the U.S. Mission to the United Nations during the second Bush administration. He was a staunch defender of former President Donald Trump and the America First doctrine. At CPAC, he declared the America First doctrine — which gave Americans more of a say in U.S. foreign policy — “here to stay.” “Previous administrations have tried to limit the American people’s participation in deciding what kind of foreign policy this country should pursue. They delegated it to unelected technocrats and career bureaucrats with the help of lobbyists and others with financial interests overseas,” he said. “The overclassification of government documents. The over-reliance of special operations forces. The over-expansion of new and overlapping federal agencies, and the over-politicized nature of intelligence all helped to take national security off the table as a matter of democratic debate,” he said. He also called out European nations reducing their contributions to NATO while building gas pipelines with Russia and making deals with China. He said the America First doctrine’s electoral popularity ensures it does not just belong to one person or party. “America First is now positioned to guide our country through the trials and tribulations of the next generation,” he said."]))

In [ ]:
bias_NN(tf.constant([".@MikePompeo is right. We need leadership wiling to fight for conservative values."]))

In [ ]:
! rm -rf "/tmp/ranking_model_dir"  # Clean up the model directory.
ranker, train_spec, eval_spec = train_and_eval_fn()
tf.estimator.train_and_evaluate(ranker, train_spec, eval_spec)